In [11]:
# #unzip data set to folders
# import zipfile
# Data = zipfile.ZipFile('./LSTM_CUDA/datasets/first-gop-debate-twitter-sentiment.zip', 'r')
# Data.extractall()
# Data.close()

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [16]:
data = pd.read_csv('./Sentiment.csv')
data = data[['text','sentiment']]

In [17]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Neutral'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

4472
6284
16986


In [22]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9293, 28) (9293, 3)
(4578, 28) (4578, 3)


In [24]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

/Users/lpk/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
 - 12s - loss: 0.8459 - accuracy: 0.6365
Epoch 2/7
 - 11s - loss: 0.7148 - accuracy: 0.6967
Epoch 3/7
 - 13s - loss: 0.6470 - accuracy: 0.7266
Epoch 4/7
 - 14s - loss: 0.6074 - accuracy: 0.7455
Epoch 5/7
 - 18s - loss: 0.5756 - accuracy: 0.7628
Epoch 6/7
 - 13s - loss: 0.5486 - accuracy: 0.7713
Epoch 7/7
 - 11s - loss: 0.5225 - accuracy: 0.7852


In [25]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.81
acc: 0.67


In [28]:
Y_validate

array([[0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [ ]:
# Negative : 0 
# Neutral  : 1
#Positive  : 2

In [31]:
pos_cnt, nerl_cnt, neg_cnt, pos_correct, nerl_correct, neg_correct = 0, 0, 0, 0,0,0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        elif np.argmax(Y_validate[x]) == 2:
            pos_correct += 1
        else:
            nerl_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    elif np.argmax(Y_validate[x]) == 2:
        pos_cnt += 1
    else:
        nerl_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("nerl_acc", nerl_correct/nerl_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 52.016129032258064 %
nerl_acc 36.231884057971016 %
neg_acc 82.57993384785006 %


In [32]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 2):
    print("positive")
else:
    print("neutral")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  204  724    5  130    6   55 1101   55   48    5  130]]
negative
